ML is great at prediction, but usually not so good at causal inference. But this paper is changing my mind! I’ll try to share a little intuition in a few tweets.

"Double/Debiased Machine Learning for Treatment and Causal Parameters"

https://arxiv.org/abs/1608.00060

First, some definitions:

$$
\begin{align}
X: & \; \mbox{pre-experiment covariates} \\
D \in \{0,1\}: & \; \mbox{treatment indicator. Depends on $X$ (in an unknown way)} \\ 
Y: & \; \mbox{outcome. Depends on $X$ and $D$} \\
\theta_0: & \; \mbox{the effect of $D$ on $Y$, this is what we want to estimate} \\
\mbox{More formally}: & \\
& Y = D \theta_0 + g(X) + U, \qquad \mathbb{E}(U|X, D) = 0 \\
& D = m(X) + V, \; \qquad \qquad \mathbb{E}(V|X) = 0 \\
\end{align}
$$

It's a long paper, but the first 7 pages really cover all the basics well. I'm new to this, so forgive any mistakes I make. And I can't do it justice here; so I'll focus only on the ideas that I recognized from other research and start with something very basic:

If we do a survey of incomes, we know that many will fail to respond. Let's assume we know, as a function of the age ($X$), the probability of responding, 𝑚(𝑋). We can then get an unbiased estimate with a weighted average, weighting by the inverse probability

$$
\left( \sum_{i \in I} \frac{1}{m(X)} \right)^{-1}  \sum_{i \in I} \frac{Y_i}{m(X)}
$$

I'll leave out the normalizing constant:

$$
\propto  \sum_{i \in I} \frac{Y_i}{m(X)}
$$

Similarly, in an AB test where the probability of being assigned to the treatment group, 𝑃(𝐷=1|𝑋) = 𝑚(𝑋) , as opposed to the control group, 𝐷=0, is a function of pre-experiment covariates $X$, we can estimate the difference as:

$$
\propto \sum_{i \in I} \frac{D_iY_i}{m(X)}-\frac{(1-D_i)Y_i}{1-m(X)}
\\
\propto \sum_{i \in I} \left( \frac{D_i}{m(X)}-\frac{(1-D_i)}{1-m(X)} \right) Y_i
\\
\mbox{multiply by $m(X)(1-m(X))$ and move it}
\\
\mbox{into the normalizing constant as it doesn't involve $Y$:}
\\
\propto  \sum_{i \in I} \big(D_i (1{-}m(X))\big) - \big((1-D_i) m(X)\big) Y_i
\\
= \left(\sum_{i \in I} \big( D_i - m(X) \big) D_i \right)^{-1} \sum_{i \in I} \big( D_i - m(X) \big) Y_i
$$


We don't know the true 𝑚(𝑋) function, but the paper shows that we can plug in an estimate from any ML method even though those methods are known to be biased.

Also, to borrow an idea from Variance Reduction, imagine we have a prediction model for Y as a function of X, then we can simply subtract this prediction from our Ys:

$$
E[Y|X] \approx g(X)
\\
= \left(\sum_{i \in I} \big( D_i - m(X) \big) D_i \right)^{-1} \sum_{i \in I} \big( D_i - m(X) \big) (Y_i - \hat{g}(X_i))
$$

With good - but potentially biased - estimates of the 𝑚(𝑋) and 𝑔(𝑋) functions, the paper shows how the estimate of θ is unbiased.